In [8]:
from env import get_connection
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from math import sqrt
from scipy import stats
from pydataset import data
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
import acquire
import prepare


In [17]:
import os

def get_zillow():
    filename = "titanic.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # read the SQL query into a dataframe
        df = pd.read_sql('SELECT yearbuilt, taxvaluedollarcnt, taxamount, bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet ,fips FROM properties_2017 JOIN propertylandusetype USING(propertylandusetypeid) WHERE propertylandusetypeid = 261; ', get_connection('zillow'))

        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename)

        # Return the dataframe to the calling code
        return df 

In [18]:
df= get_zillow()

In [19]:
df.head(100)

,yearbuilt,taxvaluedollarcnt,taxamount,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,fips
0,NaN,27516.0,NaN,0.0,0.0,NaN,6037.0
1,NaN,10.0,NaN,0.0,0.0,NaN,6037.0
2,NaN,10.0,NaN,0.0,0.0,NaN,6037.0
3,NaN,2108.0,174.21,0.0,0.0,NaN,6037.0
4,2005.0,296425.0,6941.39,4.0,2.0,3633.0,6037.0
5,NaN,124.0,NaN,0.0,0.0,NaN,6037.0
6,2011.0,847770.0,10244.94,3.0,4.0,1620.0,6037.0
7,1926.0,646760.0,7924.68,3.0,2.0,2077.0,6037.0
8,NaN,6730242.0,80348.13,0.0,0.0,NaN,6037.0
9,NaN,15532.0,248.89,0.0,0.0,NaN,6037.0
